In [1]:
# Script that receives an unclassified PSF dataset and returns the dataset with the according SEDs

import numpy as np
import tensorflow as tf
from sklearn.decomposition import PCA
import sklearn.metrics as skm

2023-07-10 10:49:50.111407: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [24]:
#PCA decomposition
dataset_path = '/Users/as274094/Documents/psf_dataset2/'
dataset_name = 'train_Euclid_res_52000_TrainStars_id_002GT_100_bins.npy'
dataset_2 = np.load(dataset_path + dataset_name, allow_pickle=True)[()] # The dataset to classify
dataset_1 = np.load('/Users/as274094/Documents/psf_dataset2/test_Euclid_res_20000_TestStars_id_002GT_100_bins.npy', allow_pickle=True)[()] # The other dataset in order to make the PCA
#dataset_2 = np.load('/Users/as274094/Documents/psf_dataset2/train_Euclid_res_52000_TrainStars_id_002GT_100_bins.npy', allow_pickle=True)[()]

# Load the stars
noisy_stars_1 = dataset_1['noisy_stars']
noisy_stars_2 = dataset_2['noisy_stars']

#should I exclude the validation stars?

fit_selection = np.concatenate((noisy_stars_1, noisy_stars_2), axis = 0)
N_components = 30

pca = PCA(n_components= N_components)
pca.fit(fit_selection.reshape(-1, 1024))
x_to_convert = pca.transform(noisy_stars_1.reshape(-1, 1024))

In [12]:
# Load model and make predictions

model_path = '/Users/as274094/GitHub/Refractored_star_classifier/tensorflow_version/best_models/config1_PCA_dataset2B30/'
classifier = tf.keras.models.load_model(model_path)

def CtoSEDarray(c_values, variance):
    sed_classes = ((c_values - 1.25) // 0.5).astype(int)
    sed_classes = np.where((c_values < 1.25) | (c_values > 7.75), 20, sed_classes)
    sed_classes = np.where((variance > 1.00), 20, sed_classes)
    return sed_classes
    
def calculate_success_rate(confusion_matrix):
    diagonal = np.trace(confusion_matrix)
    diagonal_neighbors = np.sum(np.diagonal(confusion_matrix, offset=1)) + np.sum(np.diagonal(confusion_matrix, offset=-1))
    total_classified = np.sum(confusion_matrix)
    
    success_rate = (diagonal + diagonal_neighbors) / total_classified
    return success_rate


In [25]:
# Make predictions and calculate metrics

C_pred = classifier.predict(x_to_convert, verbose = 1).reshape(-1) # Predict the scalar parameter C
class_pred = CtoSEDarray(C_pred,np.zeros_like(C_pred))

f1_mean = np.mean(skm.f1_score(dataset_1['SED_ids'], class_pred, average = None)[:13])
print('Average F1 score:', f1_mean)

confusion_matrix = skm.confusion_matrix(dataset_1['SED_ids'], class_pred)
print("\nConfusion matrix:")
print(confusion_matrix)

success_rate = calculate_success_rate(confusion_matrix)
print('\nSuccess rate:', success_rate)

625/625 [==============================] - 1s 1ms/step
Average F1 score: 0.6576688863666695

Confusion matrix:
[[ 811  642   33    5    2    0    0    0    0    0    0    0    0    1]
 [ 454  969   90   18    5    1    0    0    0    0    0    0    0    0]
 [  10  178  838  445   47    7    1    1    0    0    0    0    0    0]
 [   3   45  393  866  206   14    6    0    0    0    0    0    0    0]
 [   0    4   31  314  966  170   15    7    2    0    0    0    0    0]
 [   0    1    2   15  226 1017  289   39    8    1    1    0    0    0]
 [   0    0    0    3   33  297  860  277   27    5    1    0    0    0]
 [   0    0    0    0    3   18  161  742  607   71   10    0    0    0]
 [   0    0    0    0    2    5   68  427  886  156   10    0    0    0]
 [   0    0    0    0    0    1   12   80  710  671   44    0    0    0]
 [   0    0    0    0    0    0    1    0    3   34 1428   37    0    0]
 [   0    0    0    0    0    0    0    0    0    0   44 1527    2    0]
 [   0    0  

In [26]:
# Assign SEDs
concatenated_SEDs = np.load('concatenated_SEDs.npy', allow_pickle=True)[()]

SED_list = []
for spectral_class in class_pred:
    if spectral_class == 20:
        concat_SED = concatenated_SEDs[0] # what sould I do about the anomalies?
        print('a')
    else:
        concat_SED = concatenated_SEDs[spectral_class]
    SED_list.append(concat_SED)
SED_array = np.array(SED_list, dtype=object)

a


In [27]:
dataset_1['SEDs'] = SED_array 
dataset_1['SED_ids'] = class_pred
dataset_1['F1'] = f1_mean
dataset_1['success_rate'] = success_rate

np.save(
        dataset_path + 'assigned_' + dataset_name,
        dataset_1,
        allow_pickle=True
    )